# DeepSeek R1 模型部署指南：Ollama, vLLM 和 Transformers

DeepSeek-R1 的爆火掀起一轮本地部署大模型的热潮，尤其是 DeepSeek-R1 的 [蒸馏模型](https://github.com/deepseek-ai/DeepSeek-R1#deepseek-r1-distill-models)，因参数量小、推理速度快而备受青睐。本着跑通流程、搭搭脚手架的心态，本次部署参数量最小的 `DeepSeek-R1-Distill-Qwen-1.5B`。

大模型本地部署依赖推理引擎，目前比较流行的推理引擎有：

|推理引擎|场景|介绍|
| -- | -- |-- |
|[Ollama](https://github.com/ollama/ollama)|适合个人开发者和轻量级应用|基于 [llama.cpp](https://github.com/ggml-org/llama.cpp) 开发，支持 CPU 推理，安装简单，开箱即用，适合快速原型开发和测试|
|[vLLM](https://github.com/vllm-project/vllm)|适合高并发生产环境|支持多 GPU 并行、批处理、PagedAttention，吞吐量高，延迟低，适合大规模服务部署|
|[Transformers](https://github.com/huggingface/transformers)|适合模型研究和实验|提供完整的模型训练和推理接口，支持模型微调、量化、加速，适合研究人员和需要深度定制的场景|
|[SGLang](https://github.com/sgl-project/sglang)|适合需要复杂推理流程的场景|支持结构化输出、并行推理、流式输出，特别适合需要多轮对话和复杂推理的应用|
|[LMDeploy](https://github.com/InternLM/lmdeploy)|适合企业级部署和边缘计算|由上海人工智能实验室开发，提供完整的模型量化、加速和部署工具链，支持多种硬件平台，特别适合资源受限场景|

下面介绍如何部署 Ollama, vLLM, Transformers 这三款推理引擎，简要部署步骤见本项目 `/deploy` 目录。

## 1. Ollama

Ollama 的部署非常简单，唯一考验的就是你打开命令行的能力 (*ﾟーﾟ)

如果是 macOS 或者 Windows，从 [官网](https://ollama.com/download) 直接下载安装；如果是 Linux 系统，执行以下命令安装 Ollama：

```bsah
curl -fsSL https://ollama.com/install.sh | sh
```

验证 Ollama 是否安装成功：

```bsah
ollama --version
```

下载并运行 DeepSeek-R1-Distill-Qwen-1.5B 模型：

```bsah
ollama run deepseek-r1:1.5b
```

然后就可以在命令行使用 deepseek-r1 模型了。

<!-- 如下图：

![](./img/test_ollama.jpg)

作为 `1.5b` 模型，能有如此效果，已是相当惊人了。 -->

## 2. vLLM

vLLM 的部署也很简单，但是在个人电脑上的优化不如 Ollama，推荐有大显存 GPU 服务器的同学部署。

建议在 Ubuntu 系统安装 vLLM。如果是 Windows 电脑，可以使用 `wsl`。

我的系统配置如下：

- **系统版本**：Ubuntu 24.04.1 LTS
- **CUDA 版本**：12.6
- **显卡**：RTX 4070
- **显存**：8G

在 Ubuntu 系统下，可以用以下命令查看系统配置：

```bash
# 查看系统版本
uname -m && cat /etc/*release

# 查看 CUDA 版本
nvcc -V

# 查看 NVIDIA 显卡信息
nvidia-smi
```

下面进入正式的安装环节。

**1）安装 miniconda**

参考：https://docs.anaconda.com/miniconda/install/#quick-command-line-install

```bash
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh

~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```

然后，你需要执行以下命令激活 conda 环境：

```bash
source ~/.bashrc
```

**2）安装 jupyterlab**

```bash
conda activate base
conda install -c conda-forge jupyterlab
```

**3）为 vllm 创建虚拟环境，并绑定到 jupyterlab**

```bash
conda create --name vllm_env python=3.12
conda activate vllm_env

python -m pip install ipykernel
python -m ipykernel install --user --name vllm_env --display-name "Python (vllm_env)"
```

**4）安装 vLLM 及相关依赖**

```bash
# 安装 uv
pip install uv

# 使用 uv 安装 vllm
uv pip install vllm
uv pip install ipywidgets transformers accelerate huggingface_hub

# 验证安装是否成功
vllm --version
```

<!-- 建议像我一样分多次安装。网好随意，网络不好还一起安装我怕你哭。

```bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -i https://mirrors.aliyun.com/pypi/simple/
pip install transformers -i https://mirrors.aliyun.com/pypi/simple/
pip install accelerate huggingface_hub -i https://mirrors.aliyun.com/pypi/simple/

pip install importlib_metadata -i https://mirrors.aliyun.com/pypi/simple/
pip install vllm -i https://mirrors.aliyun.com/pypi/simple/
pip install -U ipywidgets -i https://mirrors.aliyun.com/pypi/simple/
``` -->

**5）下载 DeepSeek-R1-Distill-Qwen-1.5B 模型**

⚠ 注意：`--local-dir` 后面跟的是模型本地存储路径，请改成你的路径。

```bash
export HF_ENDPOINT=https://hf-mirror.com
huggingface-cli download --resume-download deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B --local-dir ./model/DeepSeek-R1-Distill-Qwen-1.5B
```

> **Note:** 你也可以选择从 ModelScope 下载，下载方法见本仓库的 `/model/model_download_script/models_cope.py`. 但对于本教程来说不太建议，为了后续统一，请使用 huggingface 的方法下载。

**6）启动 jupyterlab**

```bash
conda activate vllm_env
jupyter lab --ip='0.0.0.0' --port=3234 --notebook-dir=/ --no-browser --allow-root
```

打开 http://localhost:3234/lab

然后点开右上角按钮选择我们前面安装好的名为 `Python (vllm_env)` 的 Kernel

![](./img/vllm_tutorial.jpg)

**7）验证 vLLM**

遵循 DeepSeek R1 的 [官方建议](https://github.com/deepseek-ai/DeepSeek-R1/blob/main/README.md#usage-recommendations)：

1. 温度设为 0.6
2. 不写系统提示词
3. 对于数学问题，追加提示词 "Please reason step by step, and put your final answer within \boxed{}."
4. 为了确保触发思维链，在 prompt 里加 "请以 \<think\>\n 开头开始你的回答"

In [1]:
import os
import numpy as np
import torch
import torchvision
import transformers
import vllm

# 指定使用哪一块显卡
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 配置你的模型路径
MODEL_PATH = './model/DeepSeek-R1-Distill-Qwen-1.5B'

INFO 03-20 20:49:19 __init__.py:190] Automatically detected platform cuda.


In [2]:
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

torch version: 2.5.1+cu124
torchvision version: 0.20.1+cu124


In [3]:
llm = vllm.LLM(
    model=MODEL_PATH,
    gpu_memory_utilization=0.95,
    max_model_len=2048,
    tensor_parallel_size=1,
    enable_prefix_caching=True,
    max_num_batched_tokens=51200
)

INFO 03-20 20:49:25 config.py:542] This model supports multiple tasks: {'embed', 'score', 'classify', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 03-20 20:49:25 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='./model/DeepSeek-R1-Distill-Qwen-1.5B', speculative_config=None, tokenizer='./model/DeepSeek-R1-Distill-Qwen-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=./model/DeepSeek-R1-D

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-20 20:52:54 model_runner.py:1115] Loading model weights took 3.3460 GB
INFO 03-20 20:52:59 worker.py:267] Memory profiling takes 5.30 seconds
INFO 03-20 20:52:59 worker.py:267] the current vLLM instance can use total_gpu_memory (8.00GiB) x gpu_memory_utilization (0.95) = 7.60GiB
INFO 03-20 20:52:59 worker.py:267] model weights take 3.35GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 3.16GiB; the rest of the memory reserved for KV Cache is 1.06GiB.
INFO 03-20 20:52:59 executor_base.py:110] # CUDA blocks: 2474, # CPU blocks: 9362
INFO 03-20 20:52:59 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 19.33x
INFO 03-20 20:53:00 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utiliz

Capturing CUDA graph shapes: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:10<00:00,  3.45it/s]

INFO 03-20 20:53:10 model_runner.py:1562] Graph capturing finished in 10 secs, took 0.18 GiB
INFO 03-20 20:53:10 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 16.11 seconds


In [4]:
prompts = [
    '计算数学问题 (1 + 3) ^ 2 = ? 请逐步进行推理，并将你的最终答案放在 \\boxed{} 内。',
    '你将扮演一个内心火热但是表面冷淡的小偶像，请用暗含深切热爱的态度，对粉丝的晚安动态进行回复。'
]

In [5]:
# 定义采样参数
sampling_params = vllm.SamplingParams(temperature=0.6,
                                      top_p=0.95,
                                      max_tokens=8192,
                                      stop_token_ids=[151329, 151336, 151338])

# 模型推理
outputs = llm.generate(prompts, sampling_params)

len(outputs)

Processed prompts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.07it/s, est. speed input: 33.22 toks/s, output: 75.01 toks/s]


2

In [6]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt:\n{prompt}")
    print(f"Generated text:\n<think>\n{generated_text}")
    print("\n")

Prompt:
计算数学问题 (1 + 3) ^ 2 = ? 请逐步进行推理，并将你的最终答案放在 \boxed{} 内。
Generated text:
<think>
]

解题思路：
首先，计算括号内的内容，即1加3等于4。然后，对结果4进行平方运算，即4的平方等于16。因此，最终的答案是16。
</think>

首先，计算括号内的内容：

\[
1 + 3 = 4
\]

然后，对结果进行平方运算：

\[
4^2 = 16
\]

因此，最终的答案是：

\[
\boxed{16}
\]


Prompt:
你将扮演一个内心火热但是表面冷淡的小偶像，请用暗含深切热爱的态度，对粉丝的晚安动态进行回复。
Generated text:
<think>
请回复一到两句，语气要温暖，让人感到安心，不要让粉丝感到被吸引或被感染。
</think>

晚安！希望你一切安好！




## 3. Transformers

transformers 是 HuggingFace 的开源项目，它支持大量 Transformer 架构模型的推理，包括 deepseek。

In [1]:
import os
import re
import torch
import transformers

# 指定使用哪一块显卡
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 配置你的模型路径
MODEL_PATH = './model/DeepSeek-R1-Distill-Qwen-1.5B'

In [2]:
# 文本分割函数
def split_text(text):
    # 定义正则表达式模式
    pattern = re.compile(r'<think>(.*?)</think>(.*)', re.DOTALL)
    # pattern = re.compile(r'(<think>)?(.*?)</think>(.*)', re.DOTALL)
    match = pattern.search(text) # 匹配 <think>思考过程</think>回答

    if match: # 如果匹配到思考过程
        think_content = match.group(1) if match.group(1) is not None else ""
        think_content = think_content.strip() # 获取思考过程
        answer_content = match.group(2).strip() # 获取回答
    else:
        think_content = "" # 如果没有匹配到思考过程，则设置为空字符串
        answer_content = text.strip() # 直接返回回答

    return think_content, answer_content

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=False)
model = transformers.AutoModelForCausalLM.from_pretrained(MODEL_PATH,
                                                          device_map='auto',
                                                          torch_dtype=torch.bfloat16)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
prompt = '你将扮演一个内心火热但是表面冷淡的小偶像，请用暗含深切热爱的态度，对粉丝的晚安动态进行回复。'
messages = [
        {"role": "user", "content": prompt}
]

In [5]:
# 应用对话模型
input_ids = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([input_ids], return_tensors="pt").to(model.device)

# 文本生成
generated_ids = model.generate(model_inputs.input_ids,
                               attention_mask=model_inputs.attention_mask,
                               pad_token_id=tokenizer.eos_token_id,
                               temperature=0.6,
                               max_new_tokens=8192,  # 思考需要的 Token 数，设为 8K
                               top_p=0.95)

# 生成结果后处理：通过切片剔除输入部分，仅保留模型生成的内容
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# 解码：将 token id 转换为自然语言文本，并跳过特殊标记
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response = '<think>\n' + response
think_content, answer_content = split_text(response)

print(f"think: {think_content}")
print(f"answer: {answer_content}")

think: 好的，用户让我扮演一个内心火热但表面冷淡的小偶像，用暗含深切热爱的态度回复粉丝的晚安动态。首先，我需要理解用户的需求，他们希望回复既温暖又有情感，同时保持冷淡的一面。

用户可能希望回复显得真诚，但又不显太浓烈。所以，我应该用细腻的描写来传达情感，同时保持冷淡的语气。比如，用“温柔”来形容粉丝的感受，而“冷淡”则是语气上的选择。

接下来，我需要考虑如何在回复中表现出理解和关怀，同时又不显得过于情绪化。或许可以提到粉丝的陪伴，强调他们的重要性，但又不直接表达情感。

另外，用户可能希望回复带有温暖的感觉，所以可以用一些温暖的词汇，比如“温暖”、“陪伴”等，来传达情感。

最后，我应该确保回复结构清晰，情感真挚，同时保持冷淡的语气，让粉丝感受到被理解和支持。
answer: 亲爱的粉丝，你的陪伴是我最大的幸福，你总是在我最需要的时候站在我身边。你是我最忠实的依赖，也给了我无尽的温暖与关怀。你是我生命中最珍贵的财富，不会让我失望。希望你每天都能感受到我的爱与支持，永远支持我，共同书写属于我们的故事。


参考：

- vLLM GitHub: [vllm-project/vllm](https://github.com/vllm-project/vllm)
- vLLM Docs: [docs.vllm.ai](https://docs.vllm.ai/en/latest/features/reasoning_outputs.html)
- self-llm GitHub: [datawhalechina/self-llm](https://github.com/datawhalechina/self-llm/tree/master/models/DeepSeek-R1-Distill-Qwen)